In [1]:
!pip install mistralai -U -q



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install nest_asyncio -U -q
!pip install rich -U -q
!pip install python-dotenv -U -q



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from rich import print
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
from dotenv import load_dotenv 

# default directory for .env file is the current directory
# if you set .env in different directory, put the directory address load_dotenv("directory_of_.env)
load_dotenv()

True

# Basic call

In [5]:
from pydantic_ai import Agent
from pydantic import BaseModel

from pydantic_ai.models.openai import MistralModel

class CityLocation(BaseModel):
    city: str
    

model = MistralModel(model_name='mistral-large-latest', api_key=os.environ.get('MISTRAL_API_KEY'))
agent = Agent(model,result_type=CityLocation)

result = agent.run_sync('Could you give a random city? in france')
print(result.data)
#> city='London' country='United Kingdom'
print(result.cost())
#> Cost(request_tokens=56, response_tokens=8, total_tokens=64, details=None)

/var/folders/dj/wy15spl11jq1vgrmw7rltbn80000gn/T/ipykernel_91347/3267781817.py:13: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  result = agent.run_sync('Could you give a random city? in france')


CityLocation(city='Paris')

Cost(request_tokens=88, response_tokens=22, total_tokens=110, details=None)

# With function

In [6]:
from dataclasses import dataclass
from pydantic_ai import Agent, RunContext
from typing import Literal
# Define the dependencies class
@dataclass
class Deps:
    winning_number: int


model = MistralModel(model_name='mistral-large-latest', api_key=os.environ.get('MISTRAL_API_KEY'))

# Create the agent with proper typing
roulette_agent = Agent(
    model,
    deps_type=Deps,
    retries=3,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to determine if the '
        'customer has won based on the number they bet on.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(
    ctx: RunContext[Deps], square: int
) -> Literal['winner', 'loser']:
    """Check if the bet square is a winner.

    Args:
        ctx: The context containing the winning number.
        square: The number the player bet on.
    """
    return 'winner' if square == ctx.deps.winning_number else 'loser'


# Set up dependencies
winning_number = 18
deps = Deps(winning_number=winning_number)

    # Run some example bets using streaming
response = roulette_agent.run_sync(
    'Put my money on square eighteen', deps=deps
) 
print('Response', response)
print('Bet on 18:', response.data)

# async with roulette_agent.run_stream(
#     'I bet five is the winner', deps=deps
# ) as response:
#     result = await response.get_data()
#     print('Bet on 5:', result)


Response
RunResult(
    _all_messages=[
        SystemPrompt(
            content='Use the `roulette_wheel` function to determine if the customer has won based on the number 
they bet on.',
            role='system'
        ),
        UserPrompt(
            content='Put my money on square eighteen',
            timestamp=datetime.datetime(2024, 12, 8, 2, 3, 3, 352245, tzinfo=datetime.timezone.utc),
            role='user'
        ),
        ModelStructuredResponse(
            calls=[
                ToolCall(
                    tool_name='roulette_wheel',
                    args=ArgsJson(args_json='{"square": 18}'),
                    tool_id='zKwHsxmOv'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 8, 2, 3, 3, tzinfo=datetime.timezone.utc),
            role='model-structured-response'
        ),
        ToolReturn(
            tool_name='roulette_wheel',
            content='winner',
            tool_id='zKwHsxmOv',
            timestamp=datetime.datetime(2024, 12, 8, 2, 3, 4, 149581, tzinfo=datetime.timezone.utc),
            role='tool-return'
        ),
        ModelStructuredResponse(
            calls=[
                ToolCall(
                    tool_name='final_result',
                    args=ArgsJson(args_json='{"response": true}'),
                    tool_id='P9x2hZrqr'
                )
            ],
            timestamp=datetime.datetime(2024, 12, 8, 2, 3, 4, tzinfo=datetime.timezone.utc),
            role='model-structured-response'
        )
    ],
    _new_message_index=1,
    data=True,
    _cost=Cost(request_tokens=452, response_tokens=41, total_tokens=493, details=None)
)

Bet on 18: True